In [1]:
import os
import pandas as pd
import dask.dataframe as dd
import numpy as np
import glob
from tqdm import tqdm

from data_import import *

##### test cells

In [17]:
os.listdir(data_dir)

NameError: name 'data_dir' is not defined

In [ ]:
[not(i.endswith('.csv')) for i in os.listdir(data_dir)]

[False, True, True, True]

##### Data Imports

In [2]:
data_importing = OregonImport()
data_dict = data_importing.oregon_data_runner()

table management: 100%|██████████| 3/3 [00:24<00:00,  8.04s/it]
